# Unsupervised MERA

- MERA stands for Music Emotion Recognition Algorithm which was built to classify emotions from music based on both a song's lyrics and audio. The algorithm was built on the concepts of Valence and Arousal in music, which represent the spectrum of positive to negative emotions and low to high energy respectively, and Quadrants, which are representations of combinations of Valence and Arousal depending on their sign (-/+). This algorithm was a supervised task as we had a target variable.

- In this notebook, I'll be experimenting whether it's also possible to identify the target lables / Quadrants using unsupervied learning techniques. Since the dataset I'll be using for the experiment already has the expected outcome / Quadrants, I should be able to evaluate how well unsupervied techniques can be used in identitying the lables.

In [24]:
# import packages
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline

## Data loading

In [5]:
%%capture

# download data
!wget https://raw.githubusercontent.com/martinoywa/MERA/master/data/train.csv
!wget https://raw.githubusercontent.com/martinoywa/MERA/master/data/test.csv
!wget https://raw.githubusercontent.com/martinoywa/MERA/master/data/validation.csv

In [9]:
# load data
train = pd.read_csv("train.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)
test = pd.read_csv("test.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)
valid = pd.read_csv("validation.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)

# merge the data
df = pd.concat([train, test, valid])
df.head()

,,,,,quadrant,lyrics
213754,-0.682725,0.316758,Dark Tranquillity,Insanity's Crescendo,1,"Gently, hold our heads Gently, hold our heads ..."
216752,-1.400403,1.538229,Anorexia Nervosa,Stabat mater dolorosa,1,We are the Sun We are the dead stars We are th...
254391,0.815393,0.662457,Uniting Nations,Out Of Touch,0,"You're out of touch, I'm out of time But I'm o..."
266313,1.257460,1.086515,The Walkmen,Brandy alexander,0,Finally close the door You'd left open wide Lo...
283306,0.373325,-0.923151,Duke Ellington,Caravan,3,Night and stars above that shine so bright The...


In [12]:
# preprocess lyrics
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re
contractions, contractions_re = _get_contractions(contraction_dict)
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def text_preprocessing(text):
    text = text.lower() # lower text
    text = replace_contractions(text) # remove contactions
    text = "".join("".join(text).replace("\n", " ").replace("\r", " ")) # remove \n and \r
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace symbols with space
    text = re.sub(r'[0-9]+', '', text)
    text = BAD_SYMBOLS_RE.sub('', text) # replace bad characters with nothing
    text = text.strip()
    
    return text

df["lyrics"] = df["lyrics"].apply(text_preprocessing)
df.head()

,,,,,quadrant,lyrics
213754,-0.682725,0.316758,Dark Tranquillity,Insanity's Crescendo,1,gently hold our heads gently hold our heads ...
216752,-1.400403,1.538229,Anorexia Nervosa,Stabat mater dolorosa,1,we are the sun we are the dead stars we are th...
254391,0.815393,0.662457,Uniting Nations,Out Of Touch,0,you are out of touch i am out of time but i a...
266313,1.257460,1.086515,The Walkmen,Brandy alexander,0,finally close the door you would left open wid...
283306,0.373325,-0.923151,Duke Ellington,Caravan,3,night and stars above that shine so bright the...


In [13]:
# extract quadrants and lyrics
quadrants = df["quadrant"]
lyrics = df["lyrics"]

## Creating the word frequency array

In [16]:
tfidf = TfidfVectorizer()
csr_array = tfidf.fit_transform(lyrics)

In [20]:
# printing out the ompressed sparse row array
csr_array.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
# printing out the feature names
words = tfidf.get_feature_names_out()
words[:100]

array(['_______', '__________', '___________', '____________',
       '_______________', '_______________________________', 'aa', 'aaa',
       'aaaa', 'aaaaa', 'aaaaaa', 'aaaaaaa', 'aaaaaaaa',
       'aaaaaaaaaaaaaaaaaaaaaaaaah', 'aaaaaaaaaaaaaaaaaaaw',
       'aaaaaaaaaaaaaaaaah', 'aaaaaaaaaaah', 'aaaaaaaaaah', 'aaaaaaaah',
       'aaaaaaah', 'aaaaaaahhhh', 'aaaaaah', 'aaaaaahhhh', 'aaaaaahhhhh',
       'aaaaah', 'aaaaahhaaaaahhaaaaahhaaaaahhaaaaahh', 'aaaaahhhhh',
       'aaaah', 'aaaahhaaaaahh', 'aaaahhhh', 'aaaahhhhhhh',
       'aaaahhhhhhhhhh', 'aaaant', 'aaagain', 'aaah', 'aaahaaa',
       'aaahaaahaahaa', 'aaahh', 'aaahhh', 'aaahstop', 'aaand', 'aaare',
       'aaate', 'aable', 'aacacia', 'aadam', 'aagg', 'aaghhhhhhhh', 'aah',
       'aahaah', 'aahaahaah', 'aahaahaahaaa', 'aahh', 'aahhh',
       'aahhhhhhhhhhhhhhhhh', 'aaight', 'aail', 'aaja', 'aakash', 'aale',
       'aalegra', 'aalike', 'aaliyah', 'aaliyahs', 'aall', 'aalright',
       'aan', 'aanything', 'aao', 'aaooaaooh', 

Some of the data looks like a bunch of noise as they are not necessarily words. I'll leave them around for now, but these are probably worth removing

## Dimesionality reduction and clustering model

In [25]:
# TruncatedSVD is able to perform PCA on sparse arrays in csr_matrix format, such as word-frequency arrays
svd = TruncatedSVD(n_components=50)
kmeans = KMeans(n_clusters=4) # 4 clusters one for each quadrant
pipeline = make_pipeline(svd, kmeans)

In [26]:
# fit the pipeline
pipeline.fit(csr_array)

Pipeline(steps=[('truncatedsvd', TruncatedSVD(n_components=50)),
                ('kmeans', KMeans(n_clusters=4))])

In [27]:
# calculate cluster labels
labels = pipeline.predict(csr_array)

## Visualization

## Conclusion

- Other processes we can try is clustering artist, this might help us identify commonalities between them such as genres or even writing styles,